# Drone Routing Optimization Problem

## Model Initialization

In [1]:
# import Glop linear solver package
from ortools.linear_solver import pywraplp as glp
import csv
# initialize model object
mymodel = glp.Solver('Drone Routing Problem', glp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

## Parameters

### Import distance matrix

In [29]:
# import csv distance matrix as list
with open('Orders_Data.csv', newline='') as f:
    reader = csv.reader(f)
    distance = list(reader)

# remove blanks
for i in range(len(distance)):
    distance[i] = [item for item in distance[i] if item]
distance = [item for item in distance if item]
distance

[['0', '100', '81', '99', '61', '111', '16', '76', '67', '84', '102', '0'],
 ['100', '0', '97', '98', '41', '93', '97', '92', '48', '51', '9', '100'],
 ['81', '97', '0', '19', '66', '36', '66', '6', '51', '47', '92', '81'],
 ['99', '98', '19', '0', '74', '18', '84', '23', '58', '47', '92', '99'],
 ['61', '41', '66', '74', '0', '77', '56', '61', '17', '34', '42', '61'],
 ['111', '93', '36', '18', '77', '0', '97', '38', '60', '45', '86', '111'],
 ['16', '97', '66', '84', '56', '97', '0', '60', '59', '73', '98', '16'],
 ['76', '92', '6', '23', '61', '38', '60', '0', '46', '43', '88', '76'],
 ['67', '48', '51', '58', '17', '60', '59', '46', '0', '18', '45', '67'],
 ['84', '51', '47', '47', '34', '45', '73', '43', '18', '0', '45', '84'],
 ['102', '9', '92', '92', '42', '86', '98', '88', '45', '45', '0', '102'],
 ['0', '100', '81', '99', '61', '111', '16', '76', '67', '84', '102', '0']]

### Initialize Data Object

In [16]:
# model parameters
R = list(range(1, len(distance)-1)) # Nodes excluding origin & final 'dummy' node
N = list(range(len(distance))) # All nodes
M = len(distance) + 100 # Large number


## Decision Variables

In [ ]:
use_arc = [list(range(1 + N * i, 1 + N * (i+1))) for i in range(N)] 
for i in range(N):
    for j in range(N):
        use_arc[i][j] = mymodel.IntVar(0, 1, str(i+1) + "." + str(j+1))
use_arc

In [ ]:
# create position variable (lambda / u)
pos = list(range(N))
for i in range(N):
    pos[i] = mymodel.IntVar(0, N, "p_" + str(i+1))
pos

## Objective

In [ ]:
# create objective function
shortest_route = mymodel.Objective()
shortest_route.SetMinimization()
for i in range(N):
    for j in range(N):
        shortest_route.SetCoefficient(use_arc[i][j], cost[i][j])

## Constraints

### Arc constraints

In [ ]:
# first node - 1.1 + 1.2 + 1.3 + 1.4 + 1.5 = 1
#orig_node = mymodel.Constraint(1, 1)
#for i in range(N):
#    orig_node.SetCoefficient(use_arc.loc[0][i], 1)

# last node - 1.5 + 2.5 + 3.5 + 4.5 + 5.5 = 1
#last_node = mymodel.Constraint(1, 1)
#for j in range(N):
#    last_node.SetCoefficient(use_arc.loc[j][N-1], 1)

# Can't go from and to the same node - 1.1 + 2.2 + 3.3 + 4.4 + 5.5 = 1
#same_node = mymodel.Constraint(0, 0)
#for i in range(N):
#    same_node.SetCoefficient(use_arc.loc[i][i], 1)
    
# Can't go from first to dummy (1.5)
#first_to_last = mymodel.Constraint(0, 0)
#first_to_last.SetCoefficient(use_arc[N-1][0], 1)

# Can't go from dummy to first (5.1)
#last_to_first = mymodel.Constraint(0, 0)
#last_to_first.SetCoefficient(use_arc[0][N-1], 1)

In [ ]:
# Balance - Out
# 2.1 + 2.2 + 2.3 + 2.4 + 2.5 = 1

# Balance - In
# 1.2 + 2.2 + 3.2 + 4.2 + 5.2 = 1

# Flow Out

Flow_out = list(range(N))

for i in range(0, N-1): 
    Flow_out[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        Flow_out[i].SetCoefficient(use_arc[i][j], 1)
        
# Flow In

Flow_in = list(range(N))

for i in range(1, N):
    Flow_in[i] = mymodel.Constraint(1, 1)
    for j in range(N): 
        Flow_in[i].SetCoefficient(use_arc[j][i], 1)

### Position Constraints

In [ ]:
# position 1 = 1
position_1 = mymodel.Constraint(1, 1)
position_1.SetCoefficient(pos[0], 1)

# position N = N
position_N = mymodel.Constraint(1, 1)
position_N.SetCoefficient(pos[N-1], N)

# MTZ Constraint

# u[i] - u[j] + (n-1)*x[i][j] <= (n-2)

MTZ = [list(range(1 + N * i, 1 + N * (i+1))) for i in range(N)] 

for i in range(1, N):
    for j in range(1, N):
        MTZ[i][j] = mymodel.Constraint(-mymodel.infinity(), N-2)
        MTZ[i][j].SetCoefficient(use_arc[i][j], N-1)
        MTZ[i][j].SetCoefficient(pos[i], 1)
        MTZ[i][j].SetCoefficient(pos[j], -1)

In [ ]:
# Solve the model and print optimal solution
status = mymodel.Solve()                 # solve mymodel and display the solution

print('Solution Status =', status)
print('Number of variables =', mymodel.NumVariables())
print('Number of constraints =', mymodel.NumConstraints())

print('Optimal Solution:')

# The objective value of the solution.
print('Optimal Value = %.2f' % shortest_route.Value())

# Display optimal solution
for i in range(N):
    for j in range(N):
        print('From ', i + 1, ' to ', j + 1, ': ', use_arc[i][j].solution_value(), sep = '')
        
#        if i == N-1 or i == j:
#            continue
#        elif j == N-1:
 #           j = 1
 #           print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '')
 #       else:
  #          print('From node ', i + 1, ' to node ', j + 1, ': ', use_arc[j][i].solution_value(), sep = '') 